In [162]:
import pandas as pd
import numpy as np
#loading the data

train_ratings = pd.read_csv("train_ratings.csv")
no_genre = pd.DataFrame([[1, 83773.0, 1]], columns=['userId', 'movieId','rating']) #movie with genre 'none'
train_ratings = train_ratings.append(no_genre, ignore_index=True, sort=False) #insert movie at the end
print(train_ratings.tail())
val_ratings = pd.read_csv("val_ratings.csv")
test_ratings = pd.read_csv("test_ratings.csv")

          userId  movieId  rating
11946572  138493    380.0     3.0
11946573  138493    454.0     4.0
11946574  138493    500.0     4.0
11946575  138493    508.0     4.0
11946576       1  83773.0     1.0


In [151]:
import string
from gensim.parsing.preprocessing import *
from gensim.utils import simple_preprocess
movies = pd.read_csv("movies.csv")
movies.genres.replace(to_replace='(no genres listed)', value='none', inplace=True) #merge into one genre
movies.genres = movies['genres'].apply(lambda x: simple_preprocess(x))
movies.drop('title', axis=1, inplace=True)
print(movies.head())

   movieId                                             genres
0        1  [adventure, animation, children, comedy, fantasy]
1        2                     [adventure, children, fantasy]
2        3                                  [comedy, romance]
3        4                           [comedy, drama, romance]
4        5                                           [comedy]


In [163]:
train_genres = train_ratings.merge(movies, how='left', left_on='movieId', right_on='movieId')
print(train_genres.head())
# # train_genres.to_csv('train_ratings_processed.csv', index = None)
# # print(len(train_ratings.index), len(train_genres.index))
val_genres = val_ratings.merge(movies, how='left', left_on='movieId', right_on='movieId')
test_genres = test_ratings.merge(movies, how='left', left_on='movieId', right_on='movieId')

   userId  movieId  rating                                 genres
0       1      7.0     3.5                      [comedy, romance]
1       1     50.0     3.0             [crime, mystery, thriller]
2       1    151.0     4.5          [action, drama, romance, war]
3       1    160.0     3.5  [action, adventure, mystery, sci, fi]
4       1    173.0     3.0               [action, crime, sci, fi]


In [153]:
# train_genres = pd.read_csv("train_ratings_processed.csv")
# val_genres = pd.read_csv("val_ratings_processed.csv")
# test_genres = pd.read_csv("test_ratings_processed.csv")

In [154]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
train_one_hot = train_genres.join(pd.DataFrame(mlb.fit_transform(train_genres.pop('genres')), columns=mlb.classes_, index=train_genres.index))
train_one_hot.to_csv('train_ratings_one_hot.csv', index = None)
print(train_one_hot.tail())
train_one_hot.drop(train_one_hot.tail(1).index,inplace=True) #drop the row inserted earlier
print(train_one_hot.tail())

In [155]:
mlb = MultiLabelBinarizer()
val_one_hot = val_genres.join(pd.DataFrame(mlb.fit_transform(val_genres.pop('genres')), columns=mlb.classes_, index=val_genres.index))
val_one_hot.to_csv('val_ratings_one_hot.csv', index = None)

In [156]:
mlb = MultiLabelBinarizer()
test_one_hot = test_genres.join(pd.DataFrame(mlb.fit_transform(test_genres.pop('genres')), columns=mlb.classes_, index=test_genres.index))
test_one_hot.to_csv('test_ratings_one_hot.csv', index = None)